In [1]:
import numpy as np
import os
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
import graphviz
import glob
#from sklearn.metrics import mean_squared_error

In [21]:
import sys
import os
os.path.join(sys.prefix, 'lib')
xgboost/python-package
python setup.py install --use-system-libxgboost

SyntaxError: invalid syntax (<ipython-input-21-1f210b377291>, line 5)

In [2]:
df1 = pd.read_csv("https://raw.githubusercontent.com/michellegarcia2000/p-rotein/main/all_data_1.csv")
df2 = pd.read_csv("https://raw.githubusercontent.com/michellegarcia2000/p-rotein/main/all_data_2.csv")
data = pd.concat([df1,df2])

In [3]:
data.keys()

Index(['sequence', 'seq_len', 'molecular_weight', 'hydrophobicity',
       'instability_index', 'aromaticity', 'A %', 'C %', 'D %', 'E %', 'F %',
       'G %', 'H %', 'I %', 'K %', 'L %', 'M %', 'N %', 'P %', 'Q %', 'R %',
       'S %', 'T %', 'V %', 'W %', 'Y %', 'label'],
      dtype='object')

In [5]:
data["sequence"].astype("category")

0         MASREEEQRETTPERGRGAARRPPTMEDVSSPSPSPPPPRAPPKKR...
1         MKTQRDGHSLGRWSLVLLLLGLVMPLAIIAQVLSYKEAVLRAIDGI...
2         MDAQTRRRERRAEKQAQWKAANPLLVGVSAKPVNRPILSLNRKPKS...
3         MGDVEKGKKIFVQKCAQCHTVEKGGKHKTGPNLHGLFGRKTGQAPG...
4         MPKRGKKGAVAEDGDELRTEPEAKKSKTAAKKNDKEAAGEGPALYE...
                                ...                        
128394    MYIEEIILDGFKSYPTKTVIGPFHPQFNAITGLNGSGKSNVLDAIC...
128395    MYIKQIRLKGFRTYKNETTIDFTRGINCIVGFNGSGKSNILLAIEF...
128396    MVITLPKLKYALNALSPHISEETLNFHYNKHHAGYVNKLNTLIKDT...
128397    MMLNKKVVALCTLTLHLFCIFLCLGKEVRSEENGKIQDDAKKIVSE...
128398    MLNIIYVVSLILIKFIFYKECNNNNNYYLSNIELYNYKLRKRNRIL...
Name: sequence, Length: 256797, dtype: category
Categories (119847, object): ['A', 'AA', 'AAA', 'AAAA', ..., 'YYSIIPHSI', 'YYTSPMCAPARSMLLTGN', 'YYY', 'YYYYYYYYK']

In [10]:
X, y = data.iloc[:,1:-1],data.iloc[:,-1]
X["seq_len"].astype("category")
X["molecular_weight"].astype("category")
X["hydrophobicity"].astype("category")
X["instability_index"].astype("category")
X["aromaticity"].astype("category")
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [14]:
# Supported tree methods are `gpu_hist`, `approx`, and `hist`.
clf = xgb.XGBClassifier(tree_method="gpu_hist", enable_categorical=True)
clf

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              enable_categorical=True, gamma=None, gpu_id=None,
              importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method='gpu_hist',
              validate_parameters=None, verbosity=None)

In [13]:
# X is the dataframe we created in previous snippet
clf.fit(X, y)


XGBoostError: [11:09:19] /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/gbm/../common/common.h:156: XGBoost version not compiled with GPU support.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x00000001319170fe dmlc::LogMessageFatal::~LogMessageFatal() + 110
  [bt] (1) 2   libxgboost.dylib                    0x00000001319a6493 xgboost::gbm::GBTree::ConfigureUpdaters() + 531
  [bt] (2) 3   libxgboost.dylib                    0x00000001319a5e47 xgboost::gbm::GBTree::Configure(std::__1::vector<std::__1::pair<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> >, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > >, std::__1::allocator<std::__1::pair<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> >, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > > > > const&) + 1

In [ ]:
# Must use JSON/UBJSON for serialization, otherwise the information is lost.
clf.save_model("categorical-model.json")